In [2]:
import pandas as pd
btc = pd.read_csv('btc&sentimet.csv')
btc.head()

,Unnamed: 0,open,high,low,close,volume,edit_count,sentiment,neg_sentiment,tomorrow,target
0,2014-09-17 00:00:00+00:00,465.864014,468.174011,452.421997,457.334015,21056800,6.000000,-0.303405,0.650209,424.440002,0
1,2014-09-18 00:00:00+00:00,456.859985,456.859985,413.104004,424.440002,34483200,5.933333,-0.319340,0.658542,394.795990,0
2,2014-09-19 00:00:00+00:00,424.102997,427.834991,384.532013,394.795990,37919700,5.933333,-0.319830,0.658542,408.903992,1
3,2014-09-20 00:00:00+00:00,394.673004,423.295990,389.882996,408.903992,36863600,5.766667,-0.324750,0.660923,398.821014,0
4,2014-09-21 00:00:00+00:00,408.084991,412.425995,393.181000,398.821014,26580100,5.800000,-0.295682,0.644257,435.790985,1


In [3]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, min_samples_split=50, random_state=1)

train = btc.iloc[:-200]
test = btc.iloc[-200:]

predictors = ["close", "volume", "open", "high", "low", "edit_count", "sentiment", "neg_sentiment"]
model.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=50, random_state=1)

In [4]:
from sklearn.metrics import precision_score

preds = model.predict(test[predictors])
preds = pd.Series(preds, index=test.index)
precision_score(test["target"], preds)

0.49137931034482757

In [5]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["target"])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index, name="predictions")
    combined = pd.concat([test["target"], preds], axis=1)
    return combined

In [6]:
def backtest(data, model, predictors, start=1095, step=150):
    all_predictions = []

    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
    
    return pd.concat(all_predictions)

In [8]:
from xgboost import XGBClassifier

model = XGBClassifier(random_state=1, learning_rate=.1, n_estimators=200)
predictions = backtest(btc, model, predictors)

In [9]:
predictions["predictions"].value_counts()

predictions
0    287
1    248
Name: count, dtype: int64

In [10]:
precision_score(predictions["target"], predictions["predictions"])

0.5403225806451613

In [11]:
def compute_rolling(btc):
    horizons = [2,7,60,365]
    new_predictors = ["close", "sentiment", "neg_sentiment"]

    for horizon in horizons:
        rolling_averages = btc.rolling(horizon, min_periods=1).mean()

        ratio_column = f"close_ratio_{horizon}"
        btc[ratio_column] = btc["close"] / rolling_averages["close"]
        
        edit_column = f"edit_{horizon}"
        btc[edit_column] = rolling_averages["edit_count"]

        rolling = btc.rolling(horizon, closed='left', min_periods=1).mean()
        trend_column = f"trend_{horizon}"
        btc[trend_column] = rolling["target"]

        new_predictors+= [ratio_column, trend_column, edit_column]
    return btc, new_predictors

In [12]:
btc, new_predictors = compute_rolling(btc.copy())

DataError: Cannot aggregate non-numeric type: object

In [13]:
predictions = backtest(btc, model, new_predictors)

NameError: name 'new_predictors' is not defined

In [ ]:
precision_score(predictions["target"], predictions["predictions"])

In [ ]:
predictions